<a href="https://colab.research.google.com/github/KitsunekoFi/ArielACE/blob/main/NekoFi_Difusi_Stabil_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **This Notebook Made by NekoFi** ##

Upcoming Update
>Stable Diffusion 1.X Support
>Stable Diffusion 2.X Support
>LoRA Support
>Multiple LoRA Support
>IMG2IMG
>ControlNET
>T2I
>Inpaintig

Report any Problem to [NekoFi](https://www.facebook.com/arielfikru.avicenna.1?mibextid=b06tZ0)

In [ ]:

# @title ## **Menyiapkan semuanya**
!pip install -q invisible_watermark transformers accelerate safetensors diffusers xformers omegaconf imgbbpy xformers --upgrade
import torch
from transformers import AutoTokenizer
from diffusers import StableDiffusionXLImg2ImgPipeline, DiffusionPipeline, KDPM2AncestralDiscreteScheduler, StableDiffusionXLPipeline, AutoencoderKL, EulerDiscreteScheduler, EulerAncestralDiscreteScheduler
from diffusers import loaders, StableDiffusionPipeline
#from xformers.ops import MemoryEfficientAttentionFlashAttentionOp
#from diffusers.models.attention_processor import AttnProcessor
import gc
import random
from PIL import Image
import requests
from io import BytesIO
from IPython.display import display, Image
import imgbbpy
import os
import datetime

In [ ]:

# @title ## **Untuk Menampilkan Hasil**
# @markdown  **Dapatkan API Key mu di https://api.imgbb.com/  Copy dan pastekan API disini**

imgbb_api_key = "" #@param {type:"string"}
if imgbb_api_key:
   imgbb_api_key = imgbb_api_key
else:
  imgbb_api_key = "f47020c462bdb4706d579d31e604576d"

In [ ]:

# @title ## **Menyiapkan Model**

model = "Linaqruf/animagine-xl" #@param {type:"string"}
model_base = model
vae_model = "madebyollin/sdxl-vae-fp16-fix" #@param {type:"string"}
v_autoencoder = vae_model
clip_tokenizer = "openai/clip-vit-large-patch14" #@param {type:"string"}
#accelerator = "sdpa" #@param ["sdpa", "xformers", "None"]
tokenizer = AutoTokenizer.from_pretrained(clip_tokenizer)

vae = AutoencoderKL.from_pretrained(v_autoencoder, torch_dtype=torch.float16)

pipe = StableDiffusionXLPipeline.from_pretrained(
    model_base,
    vae=vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    add_watermarker=False,
    )

pipe.to("cuda")

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:

# @title ## **Main Menu**

prompts = "aesthetic, masterpiece, tohsaka rin, dojo, soft light, light color, soft, 1gifl, solo" # @param {type:"string"}
negative_prompts = "furry, lowres, realistic, pixel, loli, child" # @param {type:"string"}
scheduler = "Eular" # @param ["Eular", "Euler_A", "K_DPM2_A"]
seed = -1 # @param {type: "number"}
height = 1024 # @param {type: "number"}
width = 1024 # @param {type: "number"}
steps = 30 # @param {type: "number"}
max_steps = 100
cfg_scale = 10 # @param {type: "number"}
max_scale = 30
#batch_count = 2 # @param {type: "number"}
#max_batch_count = 10
##----------------------------##
##Tokenizer##

def split_prompt(prompt, max_tokens=77):
    tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-large-patch14")
    tokens = tokenizer(prompt, return_tensors="pt")["input_ids"][0]

    if len(tokens) <= max_tokens:
        prompt1 = prompt
        prompt2 = ""
    else:
        prompt1 = tokenizer.decode(tokens[:max_tokens])
        prompt2 = tokenizer.decode(tokens[max_tokens:])

    return prompt1, prompt2
    del tokenizer

def split_negative_prompt(negative_prompt, max_tokens=77):
    tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-large-patch14")
    tokens = tokenizer(negative_prompt, return_tensors="pt")["input_ids"][0]

    if len(tokens) <= max_tokens:
        negative_prompt1 = negative_prompt
        negative_prompt2 = ""
    else:
        negative_prompt1 = tokenizer.decode(tokens[:max_tokens])
        negative_prompt2 = tokenizer.decode(tokens[max_tokens:])

    return negative_prompt1, negative_prompt2
    del tokenizer

prompt = prompts
prompt1, prompt2 = split_prompt(prompt)
negative_prompt = negative_prompts
negative_prompt1, negative_prompt2 = split_negative_prompt(negative_prompt)
gc.collect()
torch.cuda.empty_cache()

##----------------------------##
##Additional Adjustment##

if steps > 100:
  print("Your Steps is Invalid! Use Max Steps")
  steps = max_steps

if cfg_scale > 30:
  print("Your CFG Scale is Invalid! Use Max CFG Scale")
  cfg_scale = max_scale

#if batch_count > 10:
  batch_count = max_batch_count

##----------------------------##
##Scheduler##

if scheduler == "K_DPM2_A":
  pipe.scheduler = KDPM2AncestralDiscreteScheduler.from_config(
  pipe.scheduler.config, use_karras_sigmas=True
  )
  generator = torch.Generator(device="cuda").manual_seed(seed)
elif scheduler == "Euler":
  pipe.scheduler = EulerDiscreteScheduler.from_config(
  pipe.scheduler.config
)
  generator = torch.Generator(device="cuda").manual_seed(seed)
elif scheduler == "Euler_A":
  pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(
  pipe.scheduler.config
)
  generator = torch.Generator(device="cuda").manual_seed(seed)

##----------------------------##
##Save Img##

def save_image_with_timestamped_name(base_path, image):
    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime("%Y%m%d%H%M%S")
    base_name, ext = os.path.splitext(base_path)
    result_image_path = f"{base_name}_{formatted_time}{ext}"

    image.save(result_image_path)
    return result_image_path

In [ ]:
# @title ## **⬇️ Run for Generate**

prompt = prompt1
prompt2 = prompt2
negative_prompt = negative_prompt1
negative_prompt2 = negative_prompt2

if seed == -1:
  seed = random.randint(10 ** 9, 10 ** 10 - 1)

print(f"Generating... \n\n>Prompt: '{prompt1}{prompt2}'\n>Negative Prompt: '{negative_prompts}'\n>Height: {height}\n>Width: {width}\n>Steps: {steps}\n>Scale: {cfg_scale}\n>Seed: {seed}")

image = pipe(
    prompt=prompt,
    prompt_2=prompt2,
    negative_prompt=negative_prompt,
    negative_prompt_2=negative_prompt2,
    height=height,
    width=width,
    original_size=(height,width),
    target_size=(height,width),
    crops_coords_top_left=(0,0),
    negative_original_size=(height,width),
    negative_target_size=(height,width),
    negative_crops_coords_top_left=(0,0),
    guidance_scale=cfg_scale,
    num_inference_steps=steps,
    #num_images_per_prompt=batch_count,
    ).images[0]

gc.collect()
torch.cuda.empty_cache()

result_image_path = "/content/result_image.png"
result_path = save_image_with_timestamped_name(result_image_path, image)
print("Result...")
Image(result_image_path)
client = imgbbpy.SyncClient(imgbb_api_key)
image = client.upload(file=result_path)
print(image.url)
print("Generate Done!")

Generating... 

>Prompt: 'aesthetic, masterpiece, tohsaka rin, dojo, soft light, light color, soft, 1gifl, solo'
>Negative Prompt: 'furry, lowres, realistic, pixel, loli, child'
>Height: 1024
>Width: 1024
Steps: 30
Scale: 10
>Seed: 3058424567


AttributeError: ignored